# MoM for Spherical Shell with Analytical Reference by Composing hcubature and forwardDiff 

Goal of this notebook is to perform MoM computations for the spherical shell test case with an analytical reference solution by 
1. allowing forwardDiff to operate twice on the output of hcubature to generate the MoM matrix; 
2. solve the MoM linear systems using a direct solver. 

As mesh, we use a uniform mesh of 90-degree cubes in space of spherical coordinates spanning the spherical shell source domain. 

As basis function, we use either constant functions or tensor products of constant and linear functions (other basis functions can in principle be used as well). 

We allow for collocation and Galerkin method. 

The analytical solution for magnetization vector, magnetic vector potential and magnetic flux provides intermediate reference results. 

## Import Packages

In [2]:
using ForwardDiff
using QuadGK
using HCubature 
using StaticArrays 
using LinearAlgebra
using BenchmarkTools
using Plots

In [3]:
# a point in 3D is a tuple of 3 coordinates 
# we here introduce static vectors that appear to be vital to reduce the number of allocations 
const Point3D = SVector{3,Float64};

In [22]:
zeros(Point3D,3,3)

3×3 Matrix{SVector{3, Float64}}:
 [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]  [0.0, 0.0, 0.0]

## To do (Lisette?)

1. decompose the $\phi$-component of $\vec{A}$ into cartesian basis vectors. Plot 3 cartesian components seperately as function of $(r,\theta,\phi)$ over the spherical shell; 
2. steal definitions of constants $\beta_1$ and $\gamma_1$ from Lisette's notebook;

## Section 1: Introduction 
In this notebook, we perform computations in various steps described below. 

<b>Spherical Coordinate System</b> 
We use the spherical coordinate system $\vec{r}(r,\theta,\phi)$ with [physical convection](https://en.wikipedia.org/wiki/Spherical_coordinate_system) with $0<r<\infty$, $0 \leq \theta \leq \pi$ and $0 \leq \phi < 2 \pi$. Using these coordinates, the volume of a sphere with radius $R$ is given by 

$$
\int_0^{R} \int_0^{\pi} \int_0^{2\pi}
r^2 \sin(\theta) \, dr \, d\theta \, d\phi = 4/3 \pi \, R^3 \, .  
$$

We will use this (and similar) computations as a reference for our numerical computations below. 

<b>References</b>
1. wiki on [spherical coordinates](https://en.wikipedia.org/wiki/Spherical_coordinate_system); 
2. (uses different convention, but has expression for the basis vectors) Expressions for [spherical coordinates](https://dynref.engr.illinois.edu/rvs.html);  

## Section 2: Set Constant for Test Case 

In [4]:
# inner and outer radius of spherical shell 
Ra = 1; Rb = 2;

## Section 3: Volume Integration over the Spherical Shell in Spherical Coordinates 

This section is meant to be independent of the method of moment method. 

### Section 1.3: Generate Uniform Mesh in (r_p, theta_p, phi_p) space

In [5]:
# loop over midpoints of elements 
Nr = 1; hr = (Rb-Ra)/Nr;
rmesh = Vector(Ra:hr:Rb)
Nt = 5; ht = π/Nt
tmesh = Vector(0:ht:π)
Np = 5; hp = 2*π/Np
pmesh = Vector(0:hp:2*π)
for ri in rmesh[1:end-1] 
    for tj in tmesh[1:end-1] 
        for pk in pmesh[1:end-1]
            mid = Point3D(ri+hr/2,tj+ht/2,pk+hp/2)
            # display(mid)
        end 
   end 
end 

### Section 2.3: Scalar Case 
Remark: we deliberately evaluate the integral in the dow-lower-left corner of the cell. Compared to Lisette, we take a shortcut. We rely on the difference to be small as the mesh is refined. 

In [6]:
# test integration assuming integrand for which answer can be checked analytically 
function settoone(rp)
    return 1.0
end 

# set analytical reference solution 
anal_result = 4/3*π*(Rb^3-Ra^3)

# loop over midpoints of elements 
Nr = 1; hr = (Rb-Ra)/Nr;
rmesh = Vector(Ra:hr:Rb)
Nt = 5; ht = π/Nt
tmesh = Vector(0:ht:π)
Np = 5; hp = 2*π/Np
pmesh = Vector(0:hp:2*π)
result = zeros(Nr,Nt,Np)
for (i,ri) in enumerate(rmesh[1:end-1])
    for (j,tj) in enumerate(tmesh[1:end-1])
        for (k,pk) in enumerate(pmesh[1:end-1])
            mid = Point3D(ri+hr/2,tj+ht/2,pk+hp/2)
            # display(mid)
            result[i,j,k] = hcubature(rp -> settoone(rp)*rp[1]*rp[1]*sin(rp[2]), (ri,tj,pk), (ri+hr,tj+ht,pk+hp))[1]
        end 
   end 
end 
display(sum(result)) 

display(anal_result)

29.32153143350823

29.321531433504735

In [7]:
# ugly and inefficient - fix later 
function looprprimeScalar(integrand,rmesh,tmesh,pmesh)

    Nr = length(rmesh)
    Nt = length(tmesh)
    Np = length(pmesh)
    partialresult = zeros(Nr,Nt,Np)
    for (i,ri) in enumerate(rmesh[1:end-1])
       for (j,tj) in enumerate(tmesh[1:end-1])
          for (k,pk) in enumerate(pmesh[1:end-1])
             partialresult[i,j,k] = hcubature(rp -> integrand(rp)*rp[1]*rp[1]*sin(rp[2]), (ri,tj,pk), (ri+hr,tj+ht,pk+hp))[1]
          end 
       end 
    end
    
    result = sum(partialresult)
    return result 
end 

looprprimeScalar (generic function with 1 method)

In [10]:
function settoone(rp)
    return 1.0 
end 

# set analytical reference solution 
anal_result = 4/3*π*(Rb^3-Ra^3)

Nr = 1; hr = (Rb-Ra)/Nr;
rmesh = Vector(Ra:hr:Rb)
Nt = 5; ht = π/Nt
tmesh = Vector(0:ht:π)
Np = 5; hp = 2*π/Np
pmesh = Vector(0:hp:2*π)

result = looprprimeScalar(settoone,rmesh,tmesh,pmesh)

display(result) 

display(anal_result)

29.321531433508227

29.321531433504735

### Section 3.3: Scalar Function Case

In [31]:
function looprprimeFunction(r,integrand::Function,rmesh,tmesh,pmesh)

    Nr = length(rmesh)
    Nt = length(tmesh)
    Np = length(pmesh)
    partialresult = zeros(Point3D,Nr,Nt,Np)
    for (i,ri) in enumerate(rmesh[1:end-1])
       for (j,tj) in enumerate(tmesh[1:end-1])
          for (k,pk) in enumerate(pmesh[1:end-1])
             mid = Point3D(ri+hr/2,tj+ht/2,pk+hp/2)
             # display(mid)
             partialresult[i,j,k] = hcubature(rp -> integrand(r,rp)*rp[1]*rp[1]*sin(rp[2]), (ri,tj,pk), (ri+hr,tj+ht,pk+hp))[1]
          end 
       end 
    end
    
    result = sum(partialresult)
    return result 
end 

looprprimeFunction (generic function with 1 method)

In [16]:
function scalarIntegrand(r,rprime)
    return r[1] - rprime[1]
end 

# loop over midpoints of elements 
Nr = 1; hr = (Rb-Ra)/Nr;
rmesh = Vector(Ra:hr:Rb)
Nt = 5; ht = π/Nt
tmesh = Vector(0:ht:π)
Np = 5; hp = 2*π/Np
pmesh = Vector(0:hp:2*π)

corner = Point3D(rmesh[1],tmesh[1],pmesh[1])

result = looprprimeFunction(corner,scalarIntegrand,rmesh,tmesh,pmesh)

-17.802358370345083

In [20]:
# test integration assuming integrand for which answer can be checked analytically 
function scalarIntegrand(r,rprime)
    return r[1] - rprime[1]
end 

# loop over midpoints of elements 
Nr = 2; hr = (Rb-Ra)/Nr;
rmesh = Vector(Ra:hr:Rb)
Nt = 5; ht = π/Nt
tmesh = Vector(0:ht:π)
Np = 5; hp = 2*π/Np
pmesh = Vector(0:hp:2*π)
result = zeros(Nr,Nt,Np)
for (i,ri) in enumerate(rmesh[1:end-1])
    for (j,tj) in enumerate(tmesh[1:end-1])
        for (k,pk) in enumerate(pmesh[1:end-1])
            corner = Point3D(ri,tj,pk)
            # display(mid)
            result[i,j,k] = looprprimeFunction(corner,scalarIntegrand,rmesh,tmesh,pmesh)
        end 
   end 
end 
result

2×5×5 Array{Float64, 3}:
[:, :, 1] =
 -17.8024   -17.8024   -17.8024   -17.8024   -17.8024
  -3.14159   -3.14159   -3.14159   -3.14159   -3.14159

[:, :, 2] =
 -17.8024   -17.8024   -17.8024   -17.8024   -17.8024
  -3.14159   -3.14159   -3.14159   -3.14159   -3.14159

[:, :, 3] =
 -17.8024   -17.8024   -17.8024   -17.8024   -17.8024
  -3.14159   -3.14159   -3.14159   -3.14159   -3.14159

[:, :, 4] =
 -17.8024   -17.8024   -17.8024   -17.8024   -17.8024
  -3.14159   -3.14159   -3.14159   -3.14159   -3.14159

[:, :, 5] =
 -17.8024   -17.8024   -17.8024   -17.8024   -17.8024
  -3.14159   -3.14159   -3.14159   -3.14159   -3.14159

In [33]:
function scalarIntegrand(r,rprime)
    return r - rprime
end 

# loop over midpoints of elements 
Nr = 1; hr = (Rb-Ra)/Nr;
rmesh = Vector(Ra:hr:Rb)
Nt = 5; ht = π/Nt
tmesh = Vector(0:ht:π)
Np = 5; hp = 2*π/Np
pmesh = Vector(0:hp:2*π)

corner = Point3D(rmesh[1],tmesh[1],pmesh[1])

point2 = Point3D(rmesh[2],tmesh[2],pmesh[2])

scalarIntegrand(corner, point2)

hcubature(rp -> scalarIntegrand(corner,rp)*rp[1]*rp[1]*sin(rp[2]), (rmesh[1],tmesh[1],pmesh[1]), (rmesh[2],tmesh[2],pmesh[2]))[1]

result = looprprimeFunction(corner,scalarIntegrand,rmesh,tmesh,pmesh)

3-element SVector{3, Float64} with indices SOneTo(3):
 -17.80235837035287
 -46.05815387175555
 -92.11630774351073

In [34]:
function scalarIntegrand(r,rprime)
    return r - rprime
end 

# loop over midpoints of elements 
Nr = 2; hr = (Rb-Ra)/Nr;
rmesh = Vector(Ra:hr:Rb)
Nt = 5; ht = π/Nt
tmesh = Vector(0:ht:π)
Np = 5; hp = 2*π/Np
pmesh = Vector(0:hp:2*π)
result = zeros(Point3D,Nr,Nt,Np)
for (i,ri) in enumerate(rmesh[1:end-1])
    for (j,tj) in enumerate(tmesh[1:end-1])
        for (k,pk) in enumerate(pmesh[1:end-1])
            corner = Point3D(ri,tj,pk)
            # display(mid)
            result[i,j,k] = looprprimeFunction(corner,scalarIntegrand,rmesh,tmesh,pmesh)
        end 
   end 
end 
result

2×5×5 Array{SVector{3, Float64}, 3}:
[:, :, 1] =
 [-17.8024, -46.0582, -92.1163]  …  [-17.8024, 27.6349, -92.1163]
 [-3.14159, -46.0582, -92.1163]     [-3.14159, 27.6349, -92.1163]

[:, :, 2] =
 [-17.8024, -46.0582, -55.2698]  …  [-17.8024, 27.6349, -55.2698]
 [-3.14159, -46.0582, -55.2698]     [-3.14159, 27.6349, -55.2698]

[:, :, 3] =
 [-17.8024, -46.0582, -18.4233]  …  [-17.8024, 27.6349, -18.4233]
 [-3.14159, -46.0582, -18.4233]     [-3.14159, 27.6349, -18.4233]

[:, :, 4] =
 [-17.8024, -46.0582, 18.4233]  …  [-17.8024, 27.6349, 18.4233]
 [-3.14159, -46.0582, 18.4233]     [-3.14159, 27.6349, 18.4233]

[:, :, 5] =
 [-17.8024, -46.0582, 55.2698]  …  [-17.8024, 27.6349, 55.2698]
 [-3.14159, -46.0582, 55.2698]     [-3.14159, 27.6349, 55.2698]

### Section 4.3: Vector Function Case - Regular Kernel 

### Section 5.3: Vector Function Case - Singular Kernel 

## Section 4: Given Magnetization Vector, Compute Vector Potential

<b>Introduction</b> 
Here we perform an explicit computation (intended as merely a verification step): given the $r$ and $\theta$ componenten of the magnetization vector, we compute the magetic vector potential $\vec{A}$ and verify that (a) the $\phi$-component of the magnetic vector potential matches the analytical expression and that (b) the $r$-component and $\theta$- componenten of the  magnetic vector potential are zero;

We define the kernel of integration or the vector potential density $\text{vp-dens}$ in spherical coordinates $(r,\theta,\phi)$. As in the cartesian case, the function $\text{vp-dens}$ takes the following three arguments as input:
1. the position vector of the observer (or destination) $\vec{r}(r,\theta, \phi)$;
2. the position vector of the source $\vec{r}_p(r_p,\theta_p,\phi_p)$;
3. the magnetization vector $\vec{M}$ evaluated in the coordinates of the source. 

Note that: 
1. even though magnetization vector $\vec{M}$ is $\phi$-independent, the vector potential density $\text{vp-dens}$ <b>is</b> $\phi$-dependent through $\vec{r}$ and $\vec{r}_p$. 3D-integration over the source domain (spherical shell) is thus required;
2. we would like to take advantage of the spherical symmetry of the source domain and perform integration in spherical coordinates. To deal with singularity of the kernel, we will perform integration over a set of uniform 90-degrees cubes in the 3D space $R_a <= r_p <= R_b$, $0 \leq \theta_p \leq \pi$ and $0 \leq \phi_p < 2 \, \pi$. 

The 3-component vector-valued function $\text{vp-dens}$ is given by 

$$ 
\begin{eqnarray}
\text{vp-dens}[\vec{r}, \vec{r}_p, \vec{M}] & = & 
   \text{vp-dens}[\vec{r}(r,\theta,\phi), 
                  \vec{r}_p(r_p,\theta_p,\phi_p), \vec{M}(r_p,\theta_p,\phi_p)] \\ 
   & = & \frac{\vec{M}(r_p,\theta_p,\phi_p) \times (\vec{r} - \vec{r}_p)}
        {\| \vec{r}(r,\theta,\phi) - \vec{r}_p(r_p,\theta_p,\phi_p) \|}
\end{eqnarray}
$$

Note that: 
1. in the expression above one can choose to express the cross product in either carthesian or spherical basis vectors. For reasons explaned below, we will choose cartesian basis vectors. 

The 3-component magnetic vector potential $\vec{A}(r,\theta,\phi)$ is then obtained by integrating its density over the source domain, i.e., by 

$$ 
   \vec{A}(r,\theta,\phi) = 
   \int_{R_a}^{R_b} \int_0^{\pi} \int_0^{2\pi}
   \text{vp-dens}[\vec{r}, \vec{r}_p, \vec{M}] \, 
   r^2 \sin(\theta) \, dr \, d\theta \, d\phi 
$$

Note that integration over the source domain returns a vector with 3 components (the function hcubature.jl is indeed able to integrate vector-valued functions). The resulting vector can be expressed in either carthesian or spherical coordinates.   

<b>Analytical reference solution</b>
In the test case for a spherical shell, the magnetization vector $\vec{M}(r_p,\theta_p,\phi_p)$ only has a $r$-component and a $\theta$ component. 
$$
\vec{M}(r_p,\theta_p,\phi_p) = [M_r(r,\theta), M_{\theta}(r,\theta), 0] 
$$

with 
$$
M_r(r,\theta) = -\beta_1 \cos(\theta)+2 \, \gamma_1/r^3 \cos(\theta)
$$

and 

$$
M_{\theta}(r,\theta) = \beta_1 \sin(\theta)+\gamma_1/r^3 \sin(\theta)
$$

Need to express $\vec{M}$ is carthesian basis vector in order to be able to compute the  curl with $\vec{r} - \vec{r}_p$. 

The  magnetic vector potential only has a $\phi$-component is given by 

$$
\vec{A}(r,\theta,\phi) = [0, 0, A_{\phi}(r,\theta)] 
                       = [0,0,(-\beta_1 \, r/2 +\gamma_1/r^2) \sin(\theta)]  \, . 
$$
Need to transform this to carthesian basis vector in order to be able to compare with output of hcubatute. 

<b>Pre-processing stage in function computing the vector-potential density</b>
The position vectors $\vec{r}$ and $\vec{r}_p$ can be expressed in either cartesian or spherical unit vector. For $\vec{r}$ for example, the expansion in spherical basis vectors is $\vec{r}(r,\theta,\phi) = r \, \vec{e}_r(r,\theta,\phi)$. Similarly for $\vec{r}_p(r_p,\theta_p,\phi_p) = r_p \, \vec{e}_r(r_p,\theta_p,\phi_p)$. Expression the vector of the difference $\vec{r}(r,\theta,\phi) - \vec{r}_p(r_p,\theta_p,\phi_p)$ in spherical basis vectors would require tranporting $\vec{r}_p$ to the same position as $\vec{r}_p$ (or the vice-versa). To avoid this nuisance, we express $\vec{r}$ and $\vec{r}_p$ in carthesian basis vectors. For $\vec{r}$ we have that 
$$ \vec{r}(r,\theta,\phi) = r \, \vec{e}_r(r,\theta,\phi) = 
   r \cos(\theta) \sin(\phi) \, \vec{\iota} + r \sin(\theta) \sin(\phi) \, \vec{j} 
   + r \cos(\phi) \, \vec{k}$$
and similary for $\vec{r}_p$

This can be asccomplished by a function <b>positiontoCartesian</b> (to be defined). The function <b>magnetizationtoCartesian</b> 

Need to transform vector components back to spherical basis vectors. 

In [ ]:
function positiontoCartesian(r) 
# result = SVector{3} something    
    return result   
end

In [ ]:
function magnetizationtoCartesian(M)
# result = SVector{3} something    
    return result   
end

In [ ]:
# define vector potential density 
function vp_density(r, rprime, M)
    cartr = positiontoCartesian(r) 
    cartrprime = positiontoCartesian(rprime) 
    cartM = magnetizationtoCartesian(M)
    num = cross(cartM, (cartr - cartrprime))
    denom = norm(r - rprime)^3
    result = num/denom
    return result   
end

In [ ]:
M = [0.,0.,1.]
r = [1.,0.,0.]
rp = [0.,1.,0.]
vp_density(r,rp,M)

In [ ]:
M = [0.,0.,1.]
r = [1.,0.,0.]
t = 0.5 
hcubature(rp -> vp_density(r,rp,M), (0.0, 0.0, -t), (1.0, 1.0, t))[1]

### Perform Integration with Analytical Reference Answer over Mesh in (r_p, theta_p, phi_p) - Verify answer analytically 

### Perform Integration of the vector potential density over Mesh in (r_p, theta_p, phi_p) 

## Section 5: Given Magnetization Vector, Compute Magnetic Flux

Add the computation of the curl of magnetic vector potential to previous step. Compare with analytical results for the magnetic flux. 

## Section 6: Compute Jacobian of the M-to-A Mapping 

Assume $\vec{M}$ to be constant (or linear) (but otherwise unknown) over each 3D cube in $(r_p,\theta_p,\phi_p)$  

## Section 7: Compute Jacobian of the M-to-B Mapping 

## Section 8: Given B, compute M using MoM 

## Section 9: Conclusions 